# Retrieval of classification results from Sentinel-2 L2A data

This notebook demonstrates the retrieval of region-wise time-series of aapa-mire classification data with Sentinel Hub Statistical Batch API 

Postprocessing of the raw result data is demonstsrated in notebook [postprocessing.ipynb](postprocessing.ipynb) 

Based on sentinelhub-py tutorial example: https://sentinelhub-py.readthedocs.io/en/latest/examples/batch_statistical.html#

In [2]:
%matplotlib inline

import logging
import json

import aapamires

from sentinelhub import (
    DataCollection,
    SentinelHubBatchStatistical,
    SentinelHubStatistical,
    SHConfig,
    monitor_batch_statistical_job,
)

from sentinelhub.data_utils import get_failed_statistical_requests, statistical_to_dataframe


logging.basicConfig(level=logging.INFO)


In [3]:
def build_request(input_data, inputparcels, output, evalscript, calculations,
                  resolution, timefrom, timeto, config, interval = "P1D"):
    """ Build API request """
    
    input_features = SentinelHubBatchStatistical.s3_specification(inputparcels, 
                                                                    config.aws_access_key_id, 
                                                                    config.aws_secret_access_key)

    output_s3 = SentinelHubBatchStatistical.s3_specification(output, 
                                                            config.aws_access_key_id, 
                                                            config.aws_secret_access_key)

    # Data processing
    aggregation = SentinelHubStatistical.aggregation(
        evalscript = evalscript,
        time_interval = (timefrom, timeto),
        aggregation_interval = interval,
        resolution = (resolution, resolution),
    )
    request = client.create(
        input_features = input_features,
        input_data = input_data,
        aggregation = aggregation,
        calculations = calculations,
        output = output_s3,
    )
    return request


## Set up enviroment and sentinelhub client

Import sentinelhub configuration from local file instead of default in module folder

Modify and rename [config.json.template](config.json.template) to *config.json*

To set proxies, modify and rename [proxies.ini.template](proxies.ini.template) to *proxies.ini* 

In [3]:
aapamires.setproxies()
config = aapamires.import_sh_config('config.json')
client = SentinelHubBatchStatistical(config)

In [4]:
# Object storage, bucket access must be configured as described here
# https://docs.sentinel-hub.com/api/latest/api/batch-statistical/#aws-bucket-access 
bucket_name = 's3://<mybucket>/'
parcel_path = bucket_name + "parcels/"
target_path = 'aapamire-demo/'

## Common processing parameters for all region types

In [5]:
# Input dataset
maxCloudCoverage = 0.8 # Skip all tiles where the cloud cover is greater than this value
dataType = DataCollection.SENTINEL2_L2A
input_data = [SentinelHubStatistical.input_data(dataType, maxcc = maxCloudCoverage)]

# Processing resolutions in meters
cloud_resolution = 100 
resolution = 10 
resolution_over_1km = 20 

# Temporal extent for this run
year = '2020'
timefrom = year+"-07-01T00:00:00Z"
timeto =  year+"-07-30T00:00:00Z"

## Processing of L2A data is defined in local evalscripts 


Cloud maskig <a href="./js/sumi-cloudmask-evalscript.js">sumi-cloudmask-evalscript.js</a>

Classification <a href="./js/sumi-statistics-evalscript.js">sumi-statistics-evalscript.js</a>

In [6]:
cloud_evalscript_file  = r'./js/sumi-cloudmask-evalscript.js'
classification_evalscript_file  = r'./js/sumi-statistics-evalscript.js'

with open(cloud_evalscript_file, "r") as f:
        cloud_evalscript = f.read()
with open(classification_evalscript_file, "r") as f:
        classification_evalscript = f.read() 

## First query cloud coverage for mire boundingboxes

Create request

In [9]:
cloud_parcel_file = "avo10ha_vkulkuiset_32635_shub_BoundingBoxes.gpkg" 
cloud_input_parcels = parcel_path + cloud_parcel_file
# Write results directly under given bucket without request-id
cloud_output = bucket_name + target_path + cloud_parcel_file + '/clouds/' + year + '/<ID>.json'
cloud_calculations = aapamires.cloudmask_calculations()
cloud_request = build_request(input_data, 
                              cloud_input_parcels, 
                              cloud_output, 
                              cloud_evalscript, 
                              cloud_calculations,
                              cloud_resolution, 
                              timefrom, 
                              timeto, 
                              config)
cloud_request

BatchStatisticalRequest(
  request_id=14460e13-cbd0-4c6a-8ac8-9b44eb4ea697
  created=2023-01-27 10:40:41.015193+00:00
  status=BatchRequestStatus.CREATED
  user_action=BatchUserAction.NONE
  cost_pu=0.0
  completion_percentage=0
  ...
)

Start and wait for completion

In [10]:
client.start_job(cloud_request)
monitor_batch_statistical_job(cloud_request, config=config, analysis_sleep_time=60)

INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status ANALYSING, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status ANALYSING, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job is running
Completion percentage: 100%|█████████████████████████████████████████████████████████| 100/100 [09:12<00:00,  5.53s/it]


{'id': '14460e13-cbd0-4c6a-8ac8-9b44eb4ea697',
 'status': 'DONE',
 'completionPercentage': 100,
 'lastUpdated': '2023-01-27T10:56:40.033128Z',
 'costPU': 974.3274129686966,
 'created': '2023-01-27T10:40:41.015193Z'}

## Classification of the small aapa-mires dataset (under 1km$^2$) in 10m resolution

In [12]:
parcel_file = "avo10ha_vkulkuiset_32635_shub_under1km2.gpkg" 
parcels = parcel_path + parcel_file
output = bucket_name + target_path + parcel_file + '/stats/' + year +  '/<ID>.json'
calculations = aapamires.statistics_calculations()
request = build_request(input_data, 
                        parcels, 
                        output, 
                        classification_evalscript, 
                        calculations,
                        resolution, 
                        timefrom, 
                        timeto, 
                        config)

In [13]:
client.start_job(request)
monitor_batch_statistical_job(request, config=config, analysis_sleep_time=60)

INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status ANALYSING, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status ANALYSING, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job is running
Completion percentage: 100%|█████████████████████████████████████████████████████████| 100/100 [13:49<00:00,  8.29s/it]


{'id': 'cd5eec0e-c025-46a4-b33d-91866d3288b9',
 'status': 'PROCESSING',
 'completionPercentage': 100,
 'lastUpdated': '2023-01-27T11:14:27.498921Z',
 'costPU': 8801.380502381302,
 'created': '2023-01-27T10:57:40.482053Z'}

## Classification of the large aapa-mires dataset (over 1km$^2$) in 20m resolution

In [16]:
parcel_file_over_1km="avo10ha_vkulkuiset_32635_shub_over1km2.gpkg" 
parcels_over_1km = parcel_path + parcel_file_over_1km
output_over_1km = bucket_name + target_path + parcel_file_over_1km + '/stats/' + year + '/<ID>.json'
request_over_1km = build_request(input_data, 
                                 parcels_over_1km, 
                                 output_over_1km, 
                                 classification_evalscript, 
                                 calculations,
                                 resolution_over_1km, 
                                 timefrom, 
                                 timeto, 
                                 config)

In [17]:
client.start_job(request_over_1km)
monitor_batch_statistical_job(request_over_1km, config=config, analysis_sleep_time=60)

INFO:sentinelhub.api.batch.utils:Batch job has a status CREATED, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job has a status ANALYSING, sleeping for 60 seconds
INFO:sentinelhub.api.batch.utils:Batch job is running
Completion percentage: 100%|█████████████████████████████████████████████████████████| 100/100 [21:00<00:00, 12.60s/it]


{'id': 'c02a17a2-6a55-4825-b7fc-34d28a67bacf',
 'status': 'DONE',
 'completionPercentage': 100,
 'lastUpdated': '2023-01-27T11:42:17.325158Z',
 'costPU': 4470.507035184571,
 'created': '2023-01-27T11:19:33.856842Z'}


## Results can be retrieved from AWS S3 bucket with external tools (s3cmd, aws-cli) 

For example aws --profile myprofile s3 sync s3://\<mybucket\> .

Postprocessing of the complete dataset is demonstrated in [postprocessing.ipynb](postprocessing.ipynb)
